<a href="https://colab.research.google.com/github/piyush202002/Retail-Analytics-Case-Study/blob/main/Retail%20Analytics%20Case%20Study%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retail Analytics Case Study

Q1.Problem statement
Write a query to identify the number of duplicates in "sales_transaction" table. Also, create a separate table containing the unique values and remove the the original table from the databases and replace the name of the new table with the original name.

Hint:
Use the “Sales_transaction” table.
There will be two resulting tables in the output. First, the table where the count of duplicates will be identified and in the second table we can check if the duplicates were removed or not by selecting the whole table.

In [ ]:
select transactionID as TransactionID,count(*) from sales_transaction
group by transactionID
Having count(*)>1;
CREATE TABLE sales_transaction_unique AS(
SELECT DISTINCT *
FROM sales_transaction);
drop table sales_transaction;
Alter table sales_transaction_unique
Rename sales_transaction;
select * from sales_transaction;


Q2.Problem statement
Write a query to identify the discrepancies in the price of the same product in "sales_transaction" and "product_inventory" tables. Also, update those discrepancies to match the price in both the tables.


Hint:

Use the "sales_transaction" and the "product_inventory" tables.
There will be two resulting tables in the output. First, the table where the discrepancies will be identified and in the second table we can check if the discrepancies were updated or not.


In [ ]:
select sales_transaction.TransactionID,sales_transaction.price as TransactionPrice,product_inventory.price as InventoryPrice
from sales_transaction
inner join product_inventory
on sales_transaction.productid=product_inventory.productid
where sales_transaction.price!=product_inventory.price;
update sales_transaction
set Price='93.12'
where Price='9312';
select*from sales_transaction;

Q3.Problem statement
Write a SQL query to identify the null values in the dataset and replace those by “Unknown”.


Hint:

Use the customer_profiles table.
Identify the columns which contains null values and count the number of cells containing null values. Update those values with “unknown” and showcase the changes that the query has created.

In [ ]:
select count(*) from customer_profiles
where location is null;
update customer_profiles
set location='Unknown'
where location is null;
select * from customer_profiles;

Q4.Problem statement
Write a SQL query to summarize the total sales and quantities sold per product by the company.


(Here, the data has been already cleaned in the previous steps and from here we will be understanding the different types of data analysis from the given dataset.)

Hint:

Use the “Sales_transaction” table.
The resulting table will display the total quantity purchased by the customers and the total sales done by the company to evaluate the product performance.
Return the result table in descending order corresponding to Total Sales Column.

In [ ]:
select productID,sum(quantitypurchased)TotalUnitsSold,sum(quantitypurchased*price) as TotalSales
from sales_transaction
group by productId
order by TotalSales desc;

Q5.Problem statement
Write a SQL query to count the number of transactions per customer to understand purchase frequency.

Hint:
Use the “Sales_transaction” table.
The resulting table will be counting the number of transactions corresponding to each customerID.
Return the result table ordered by NumberOfTransactions in descending order.

In [ ]:
SELECT customerid,
       COUNT(*) AS NumberOfTransactions
FROM sales_transaction
GROUP BY customerid
ORDER BY NumberOfTransactions DESC;


Q6.Problem statement
Write a SQL query to evaluate the performance of the product categories based on the total sales which help us understand the product categories which needs to be promoted in the marketing campaigns.


Hint:

Use the “Sales_transaction” and "product_inventory" table.
The resulting table must display product categories, the aggregated count of units sold for each category, and the total sales value per category.
Return the result table ordering by TotalSales in descending order.

In [ ]:
select product_inventory.category as Category,sum(quantitypurchased)TotalUnitsSold,
sum(sales_transaction.price*quantitypurchased)TotalSales from product_inventory
inner join sales_transaction
on product_inventory.productid=sales_transaction.productid
group by product_inventory.category
order by TotalSales desc;



Q7.Problem statement
Write a SQL query to find the top 10 products with the highest total sales revenue from the sales transactions. This will help the company to identify the High sales products which needs to be focused to increase the revenue of the company.

Hint:
Use the “Sales_transaction” table.
The resulting table should be limited to 10 productIDs whose TotalRevenue (Product of Price and QuantityPurchased) is the highest.
Return the result table ordering by TotalRevenue in descending order.

In [ ]:
select productid,sum(quantitypurchased*price)as TotalRevenue from sales_transaction
group by productid
order by totalRevenue desc
limit 10;

Q8.Problem statement
Write a SQL query to find the ten products with the least amount of units sold from the sales transactions, provided that at least one unit was sold for those products.


Hint:

Use the “Sales_transaction” table.
The resulting table should be limited to 10 productIDs whose TotalUnitsSold (sum of QuantityPurchased) is the least. (The limit value can be adjusted accordingly)
Return the result table ordering by TotalUnitsSold in ascending order.


In [ ]:
select productid as Productid,sum(quantitypurchased)as TotalUnitsSold from sales_transaction
group by productid
having TotalUnitsSold>=1
order by TotalUnitsSold
limit 10;

Q9.Problem statement
Write a SQL query to identify the sales trend to understand the revenue pattern of the company.

Hint:

Use the “sales_transaction” table.
The resulting table must have DATETRANS in date format, count the number of transaction on that particular date, total units sold and the total sales took place.
Return the result table ordered by datetrans in descending order.

In [ ]:
select transactiondate as DATETRANS ,count(transactionid)Transaction_count,sum(quantitypurchased)TotalUnitsSold,
sum(quantitypurchased*price)as TotalSales from sales_transaction
group by DATETRANS
order by DATETRANS  desc;


Q10.Problem statement
Write a SQL query to understand the month on month growth rate of sales of the company which will help understand the growth trend of the company.

Hint:

Use the “sales_transaction” table.
The resulting table must extract the month from the transactiondate and then the Month on month growth percentange should be calculated. (Total sales present month - total sales previous month/ total sales previous month * 100)
Return the result table ordering by month.

In [ ]:
with cte1 as(
select extract(month from transactiondate) as month,sum(quantitypurchased*price) as total_sales,
lag(sum(quantitypurchased*price),1) over(order by extract(month from transactiondate)) as previous_month_sales
from sales_transaction
group by month
order by month
)
select *,((total_sales-previous_month_sales) /previous_month_sales)*100 as
mom_growth_percentage
from cte1;



Q11.Problem statement
Write a SQL query that describes the number of transaction along with the total amount spent by each customer which are on the higher side and will help us understand the customers who are the high frequency purchase customers in the company.

Hint:

Use the “sales_transaction” table.
The resulting table must have number of transactions more than 10 and TotalSpent more than 1000 on those transactions by the corresponding customers.
Return the result table on the “TotalSpent” in descending order.

In [ ]:
with cte as(
select customerid as CustomerID,count(transactionid)NumberofTransactions,
sum(quantitypurchased*price)as TotalSpent from sales_transaction
group by customerid
)
select CustomerID, NumberofTransactions,TotalSpent from cte
where NumberofTransactions>10 and TotalSpent>1000
order by  totalspent desc;

Q12.Problem statement
Write a SQL query that describes the number of transaction along with the total amount spent by each customer, which will help us understand the customers who are occasional customers or have low purchase frequency in the company.

Hint:

Use the “Sales_transaction” table.
The resulting table must have number of transactions less than or equal to 2 and corresponding total amount spent on those transactions by related customers.
Return the result table of “NumberOfTransactions” in ascending order and “TotalSpent” in descending order.

In [ ]:
select customerid as CustomerID,count(transactionid)NumberOfTransactions, sum(quantitypurchased*price)TotalSpent from sales_transaction
group by customerid
having NumberOfTransactions<=2
order by NumberOfTransactions ,TotalSpent desc

Q13.Problem statement
Write a SQL query that describes the total number of purchases made by each customer against each productID to understand the repeat customers in the company.


Hint:

Use the “Sales_transaction” table.
The resulting table must have "CustomerID", "ProductID" and the number of times that particular customer have purchases the product.
The number of times the customer has purchased should be more than once.
Return the result table in descending order corresponding to the TimesPurchased column.

In [ ]:
select customerid as CustomerId,productid as ProductID, count(transactionid)TimesPurchased from sales_transaction
group by customerid,productid
having TimesPurchased>1
order by TimesPurchased desc;

Q14.Problem statement
Write a SQL query that describes the duration between the first and the last purchase of the customer in that particular company to understand the loyalty of the customer.

Hints:
Use the "Sales_transaction" table.
The DATE column will be majorly in use in the question and the TransactionDate column in Sales_transaction is in text format. Thus, the format of the TransactionDate column should be changed.
The resulting table must have the first date of purchase, the last date of purchase and the difference between the first and the last date of purchase.
The difference between the first and the last date of purchase should be more than 0.
Return the table in descending order corresponding to DaysBetweenPurchases.

In [ ]:
select customerid as CustomerID,min(transactiondate)FirstPurchase,max(transactiondate)LastPurchase,
datediff(max(transactiondate),min(transactiondate))DaysBetweenPurchases from sales_transaction
group by customerid
having DaysBetweenPurchases>0
order by DaysBetweenPurchases desc;